In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
from parameters_dict import physics_parameters
from exp_analysis_class import exp_analysis
from exp_analysis import gamma_heavy, gamma_heavy_integrated

In [3]:
physics_parameters['heavy']

{'m4_limits': (0.005, 1),
 'mz_limits': (0.005, 10),
 'm4_scan': [0.01, 0.1, 0.5],
 'mz_scan': [0.01, 0.1, 1, 5],
 'alpha_dark': 0.4,
 'Umu4': 2.2e-07,
 'epsilon2': 0.00046,
 'alpha_em': 0.0072992700729927005,
 'span_2d': 9.945025,
 'Vmu4_alpha_epsilon2': 2.9547445255474454e-13}

In [4]:
case = 'heavy'
produce_samples = False
save_folder = ('../fig/control_checks/comparison_with_truth/' + case + '/')
!mkdir -p $save_folder
my_exp_analysis = exp_analysis(case)

/bin/bash: port_used: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `BASH_FUNC_port_used'
/bin/bash: wait_until_port_used: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `BASH_FUNC_wait_until_port_used'
/bin/bash: module: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `BASH_FUNC_module'
/bin/bash: find_port: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `BASH_FUNC_find_port'
/bin/bash: random_number: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `BASH_FUNC_random_number'
/bin/bash: helmod: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `BASH_FUNC_helmod'
/bin/bash: source_helpers: line 1: syntax error: unexpected end of file
/bin/bash: error importing function definition for `BASH_FUNC_source_h

In [5]:
my_exp_analysis.load_df_base(1000000)

In [22]:
#generation part
if produce_samples:
    for m4, mz_prime in itertools.product(physics_parameters[case]['m4_scan'], physics_parameters[case]['mz_scan']):
        print(m4, mz_prime)
        dark_gen_run = f'cd ..; python dark_gen.py --M4 {m4} --mzprime {mz_prime} '\
                       f'--UMU4 {physics_parameters[case]["Umu4"]} '\
                       f'--alpha_dark {physics_parameters[case]["alpha_dark"]} '\
                       f'--epsilon2 {physics_parameters[case]["epsilon2"]} '\
                       f'--neval 100000 --noplot --hierarchy {case}_mediator'
        stream = os.popen(dark_gen_run)
        print(stream.read())

0.01 0.01
ERROR! Mass spectrum not allowed.

0.01 0.1
Model used 3+1
6221586.764186033

0.01 1
Model used 3+1
881.5054971076404

0.01 5
Model used 3+1
1.0299001232318477

0.1 0.01
Model used 3+1
623528941116.2891

0.1 0.1
ERROR! Mass spectrum not allowed.

0.1 1
Model used 3+1
891.3190218853877

0.1 5
Model used 3+1
0.9406405991404354

0.5 0.01
Model used 3+1
399662123.89109576

0.5 0.1
Model used 3+1
7964821.9990316015

0.5 1
Model used 3+1
59.38789528499492

0.5 5
Model used 3+1
0.032270336522846534



In [23]:
aux_gamma = []
aux_sigma = []
aux_n_evt = []
for m4 in physics_parameters[case]['m4_scan']:
    aux_gamma.append([])
    aux_sigma.append([])
    aux_n_evt.append([])
    for mz in physics_parameters[case]['mz_scan']:
        if ((case == 'heavy') and (m4 >= mz)) or ((case == 'light') and (m4 <= mz)):
                aux_gamma[-1].append(0)
                aux_sigma[-1].append(0)
                aux_n_evt[-1].append(0)
                continue
        else:
            df = my_exp_analysis.load_df(m4, mz)
            aux_gamma[-1].append(df['weight_decay', ''][df['lead', '']].sum())
            aux_sigma[-1].append(df['adjusted_weight', ''][df['lead', '']].sum())
            aux_n_evt[-1].append(df['actual_weight', ''].sum())

aux_gamma = np.array(aux_gamma)
aux_sigma = np.array(aux_sigma)
aux_n_evt = np.array(aux_n_evt)

In [31]:
# produce the gamma profile
material = 'lead'
mask = my_exp_analysis.df_base[material, '']
m4_values = my_exp_analysis.df_base['m4', ''][mask].values
mz_values = my_exp_analysis.df_base['mzprime', ''][mask].values
weight_decay_values = my_exp_analysis.df_base['weight_decay', ''][mask].values

m4_span = physics_parameters[case]['m4_scan']
mz_span = np.linspace(*physics_parameters[case]['mz_limits'], 18)
grid_to_eval = np.stack(np.meshgrid(m4_span, mz_span, indexing='ij'), axis=-1)
df_values = np.stack([m4_values, mz_values], axis=-1)

smoothing_pars = [0.05, 0.1]
this_kde_weights = exp_analysis.kde_Nd_weights(grid_to_eval, df_values, smoothing=smoothing_pars)
final_weights = this_kde_weights * weight_decay_values[:, np.newaxis, np.newaxis]
gamma_grid = np.sum(final_weights, axis=0)
gamma_std_grid = np.sqrt(np.sum(final_weights**2, axis=0))

for i, m4 in enumerate(physics_parameters[case]['m4_scan']):
    print(i, m4)
    plt.plot(physics_parameters[case]['mz_scan'], aux_gamma[i], 'o', label='Generated without scanning')
    gamma_est = gamma_grid[i]
    gamma_std = gamma_std_grid[i]
    plt.plot(mz_span, gamma_est, label=f'kde 2d, $k_{{m_4}}$={smoothing_pars[0]:.2g}, $k_{{m_Z}}$={smoothing_pars[1]:.2g}')
    plt.fill_between(mz_span, gamma_est-gamma_std, gamma_est+gamma_std, alpha=0.25, color=plt.gca().lines[-1].get_color(), interpolate=True)

    bins = 20
    bin_width_m4 = 0.01
    this_mask = (m4_values > (m4-bin_width_m4/2)) & (m4_values < (m4+bin_width_m4/2))
    bin_range = [mz_values[this_mask].min(), mz_values[this_mask].max()]
    
    bin_size = (bin_range[1] - bin_range[0])/bins * bin_width_m4
    out = plt.hist(mz_values[this_mask], bins=bins, range=bin_range,
                weights=weight_decay_values[this_mask]/bin_size,
                histtype='step', label='histogram')

    plt.legend(frameon=False, loc='upper right')
    plt.xlim(mz_values[this_mask].min(), mz_values[this_mask].max())
    plt.xlabel(r"$M_Z'$ [GeV]")
    plt.ylabel('Decay rate [GeV]')
    plt.yscale('log')
#     plt.xscale('log')
#     plt.ylim(0, 1e-12)
    # plt.xlim(bin_range)
    plt.title(f'N random = {my_exp_analysis.n_evt}\n{case} mediator, $m_4$ = {m4} GeV, only {material}')
    plt.tight_layout()
#     break
    plt.savefig(save_folder+f'gamma_{material}_nevt_{my_exp_analysis.n_evt}_m4_{m4}.png', dpi=250)
    plt.close()

0 0.01
1 0.1
2 0.5


In [ ]:
# produce the sigma profile
adjusted_weight = df_base['adjusted_weight', ''][mask].values
final_weights = this_kde_weights * adjusted_weight[:, np.newaxis, np.newaxis]
sigma_grid = np.sum(final_weights, axis=0)
sigma_std_grid = np.sqrt(np.sum(final_weights**2, axis=0))

for i, m4 in enumerate(pars[case]['m4']):
    plt.plot(pars[case]['mz_prime'], aux_sigma[i], 'o', label='true function')
    sigma_est = sigma_grid[i]
    sigma_std = sigma_std_grid[i]
    plt.plot(mz_span, sigma_est, label=f'kde 2d, $k_{{m_4}}$={smoothing_pars[0]:.2g}, $k_{{m_Z}}$={smoothing_pars[1]:.2g}')
    plt.fill_between(mz_span, sigma_est-sigma_std, sigma_est+sigma_std, alpha=0.25, color=plt.gca().lines[-1].get_color(), interpolate=True)

    bins = 20
    bin_width_m4 = 0.02
    this_mask = (m4_values > (m4-bin_width_m4/2)) & (m4_values < (m4+bin_width_m4/2))
    bin_range = [mz_values[this_mask].min(), mz_values[this_mask].max()]
    bin_span = (bin_range[1] - bin_range[0])
    bin_span_m4 = (m4_values.max() - m4_values.min())
    bin_width = bin_span/bins
    additional_weight = bin_span * bin_span_m4 / bin_width / bin_width_m4
    print(mz_values[this_mask], additional_weight, adjusted_weight[this_mask])
    print(bins, bin_range)
    out = plt.hist(mz_values[this_mask], bins=bins, range=bin_range,
                weights=adjusted_weight[this_mask]*additional_weight,
                histtype='step', label='histogram')

    plt.legend(frameon=False, loc='upper right')
    plt.xlabel(r"$M_Z'$ [GeV]")
    plt.ylabel('Cross section [GeV$^{-2}$]')
    # plt.yscale('log')
    plt.ylim(0, 1.2*max(sigma_est))
    # plt.xlim(bin_range)
    plt.title(f'N random = {n_evt}\n{case} mediator, $m_4$ = {m4} GeV, only {material}')
    plt.tight_layout()
    plt.savefig(save_folder+f'sigma_{material}_nevt_{n_evt}_m4_{m4}.png', dpi=250)
    plt.close()

In [ ]:
# produce the n_evt profile at different stages in the selection
actual_weight = df_base['actual_weight', ''][mask].values
final_weights = this_kde_weights * actual_weight[:, np.newaxis, np.newaxis]
n_event_grid = np.sum(final_weights, axis=0)
n_event_std_grid = np.sqrt(np.sum(final_weights**2, axis=0))

ctau = 25
for mask in ['trivial', 'selected', f'decay_in_tpc_{ctau}', f'total_selected_{ctau}']:
    for i, m4 in enumerate(pars[case]['m4']):
        plt.plot(pars[case]['mz_prime'], aux_n_evt[i], 'o', label='true function')
        n_event_est = n_event_grid[i]
        n_event_std = n_event_std_grid[i]
        plt.plot(mz_span, n_event_est, label=f'kde 2d, $k_{{m_4}}$={smoothing_pars[0]:.2g}, $k_{{m_Z}}$={smoothing_pars[1]:.2g}')
        plt.fill_between(mz_span, n_event_est-n_event_std, n_event_est+n_event_std, alpha=0.25, color=plt.gca().lines[-1].get_color(), interpolate=True)

        bins = 20
        bin_width_m4 = 0.02
        this_mask = (m4_values > (m4-bin_width_m4/2)) & (m4_values < (m4+bin_width_m4/2))
        bin_range = [mz_values[this_mask].min(), mz_values[this_mask].max()]
        bin_span = (bin_range[1] - bin_range[0])
        bin_span_m4 = (m4_values.max() - m4_values.min())
        bin_width = bin_span/bins
        additional_weight = bin_span * bin_span_m4 / bin_width / bin_width_m4
        out = plt.hist(mz_values[this_mask], bins=bins, range=bin_range,
                    weights=actual_weight[this_mask]*additional_weight,
                    histtype='step', label='histogram')

        plt.legend(frameon=False, loc='upper right')
        plt.xlabel(r"$M_Z'$ [GeV]")
        plt.ylabel('Expected number of entries')
        # plt.yscale('log')
        # plt.ylim(0, 1e-7)
        # plt.xlim(bin_range)
        plt.title(f'N random = {n_evt}\n{case} mediator, $m_4$ = {m4} GeV, only {material}')
        plt.tight_layout()
        plt.savefig(save_folder+f'mask_{mask}_nevt_{n_evt}_m4_{m4}.png', dpi=250)
        plt.close()